In [1]:
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, Dense, Lambda
from tensorflow.keras.utils import to_categorical
import tensorflow
import pandas as pd
import numpy as np

In [2]:
normal_model = load_model("my_model.h5")
test = pd.read_csv('./input/train.csv')
print("Data are Ready!!")
print(f"Testing data size is {test.shape}")

Metal device set to: Apple M2

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-08-09 00:17:33.364250: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-08-09 00:17:33.364387: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Data are Ready!!
Testing data size is (42000, 785)


In [3]:
X_test = test.drop(['label'], 1).values
y_test = test['label'].values

/var/folders/v3/ybc9hrrd0yg6d_zr2sbckrtm0000gn/T/ipykernel_7121/3112465331.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X_test = test.drop(['label'], 1).values


In [4]:
_, X_test, _, y_test = train_test_split(X_test, y_test, test_size=0.1, random_state=0)

In [5]:
X_test = X_test / 255.0
X_test = X_test.reshape(-1,28,28,1)

y_test = to_categorical(y_test)

print(f"Label size {y_test.shape}")

Label size (4200, 10)


In [6]:
def random_multiplier(shape):
    return tensorflow.random.uniform(shape, minval=0.99, maxval=1.01)

class RandomMultiplierLayer(tensorflow.keras.layers.Layer):
    def call(self, inputs):
        random_factors = random_multiplier(inputs.shape[1:])
        return inputs * random_factors

# 커스텀 레이어 추가하여 새 모델 구축
error_model = Sequential()
for layer in normal_model.layers:
    error_model.add(layer)  # 기존 레이어 추가
    # if isinstance(layer, Conv2D) or isinstance(layer, Dense):
    error_model.add(RandomMultiplierLayer())  # 랜덤한 값을 곱하는 레이어 추가

# 컴파일 및 요약
error_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
test_error_loss, test_error_accuracy = error_model.evaluate(X_test, y_test, verbose=2)
test_normal_loss, test_normal_accuracy = normal_model.evaluate(X_test, y_test, verbose=2)
print(f"Test normal model accuracy: {test_normal_accuracy} Test error model accuracy: {test_error_accuracy}")

2023-08-09 00:17:35.462866: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-08-09 00:17:35.721323: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


132/132 - 2s - loss: 0.0145 - accuracy: 0.9950 - 2s/epoch - 15ms/step


2023-08-09 00:17:37.576776: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


132/132 - 1s - loss: 0.0145 - accuracy: 0.9950 - 1s/epoch - 11ms/step
Test normal model accuracy: 0.9950000047683716 Test error model accuracy: 0.9950000047683716


In [8]:
def save_weights_to_text(model, file_path):
    with open(file_path, 'w') as file:
        for layer in model.layers:
            weights = layer.get_weights()
            if len(weights) > 0:
                weights_text = [str(w.tolist()) for w in weights]
                weights_str = "\n".join(weights_text)
                file.write(f"Layer: {layer.name}\n")
                file.write(weights_str + "\n\n")

In [9]:
weights_file_path = './model_weights.txt'
save_weights_to_text(normal_model, weights_file_path)

In [10]:
weights_file_path = './error_model_weights.txt'
save_weights_to_text(error_model, weights_file_path)